## Script Recording with Checksum Validation

### What This Does

This system records Python scripts to a text file with integrity protection. It ensures the script hasn't been tampered with before execution.

### How?

1. Take Python script as text
2. Calculate SHA-256 hash (checksum)
3. Base64-encode the script
4. Save: checksum + newline + encoded_script
5. To run: decode, verify checksum, execute if valid

### Validation Process
- **Read**: Load checksum and encoded script from file
- **Decode**: Convert base64 back to original Python code
- **Verify**: Calculate checksum of decoded script
- **Compare**: Check if calculated checksum matches stored checksum
- **Execute**: Only run script if checksums match (integrity confirmed)

In [ ]:
import hashlib
import base64

def startScriptRecording(script_text):
    script_bytes = script_text.encode('utf-8')
    checksum = hashlib.sha256(script_bytes).hexdigest()
    encoded_script = base64.b64encode(script_bytes).decode('utf-8')
    return encoded_script, checksum

def writeScriptWithChecksum(filename, encoded_script, checksum):
    with open(filename, 'w') as f:
        f.write(checksum + '\n')
        f.write(encoded_script + '\n')
    print(f"✓ Saved to {filename}: checksum + encoded script")

def readScriptWithChecksum(filename):
    with open(filename, 'r') as f:
        lines = f.readlines()
    checksum = lines[0].strip()
    encoded_script = ''.join(line.strip() for line in lines[1:])
    print(f"✓ Loaded from {filename}: checksum + encoded script")
    return encoded_script, checksum

def validateAndRunScript(encoded_script, checksum):
    # 1. Decode the base64 script back to original
    script_bytes = base64.b64decode(encoded_script.encode('utf-8'))
    
    # 2. Calculate checksum of the decoded script
    calculated_checksum = hashlib.sha256(script_bytes).hexdigest()
    
    # 3. Compare with stored checksum
    original_script = script_bytes.decode('utf-8')
    
    if calculated_checksum == checksum:
        exec(script_bytes.decode('utf-8'))
        print(f"✓ Valid & executed: {original_script}")
        return True
    else:
        print("✗ Invalid: checksum mismatch")
        return False

# Example usage
script = 'print("Hello from recorded script;")'
print(f"Script: {script}")

encoded_script, checksum = startScriptRecording(script)
print(f"Checksum: {checksum}")

# Write checksum and script to one file
writeScriptWithChecksum('recorded_script_with_checksum.txt', encoded_script, checksum)

# Read and validate from the same file
loaded_script, loaded_checksum = readScriptWithChecksum('recorded_script_with_checksum.txt')

validateAndRunScript(loaded_script, loaded_checksum)

Script: print("Hello from recorded script;")
Checksum: 22d808df71c2ae3534eaace31581c4b42170399d82ff6c2c5fcfb26227e4f51e
✓ Saved to recorded_script_with_checksum.txt: checksum + encoded script
✓ Loaded from recorded_script_with_checksum.txt: checksum + encoded script
Hello from recorded script;
✓ Valid & executed: print("Hello from recorded script;")


True